In [1]:
%load_ext dotenv
%dotenv

In [2]:
import os
import csv
import json
from google.cloud import pubsub_v1
from tqdm import tqdm

In [3]:
project_id = os.environ['PROJECT_ID']
topic_id = os.environ['TOPIC_ID']

In [4]:
# Initialize the Pub/Sub publisher client
publisher = pubsub_v1.PublisherClient()
topic_path = publisher.topic_path(project_id, topic_id)

In [5]:
file_path = "data/review_written.csv"

In [6]:
total_rows = 0

with open(file_path, "r", newline="", encoding="utf-8") as csvfile:
    reader = csv.DictReader(csvfile)
    file_row_count = sum(1 for row in reader)
    total_rows += file_row_count

print(f"Total number of rows to publish: {total_rows}")

Total number of rows to publish: 1135


In [7]:
with tqdm(total=total_rows, desc="Publishing messages") as pbar:
    with open(file_path, "r", newline="", encoding="utf-8") as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            message_json = json.dumps(row)
            message_data = message_json.encode("utf-8")
            
            future = publisher.publish(topic_path, data=message_data)
            future.result()
            
            pbar.update(1)

Publishing messages: 100%|█████████████████████████████████████████████████████████████████████| 1135/1135 [01:01<00:00, 18.33it/s]
